# Introduction

Here is a brief intro to some of the basics of using PySpark. You'll need to have Hadoop, Spark, and an ssh server available on your machine. I'm running on a laptop, so I don't normally have the ssh server or Hadoop running. If you aren't used to Linux, you can start ssh with a command like this (you may need to sudo these):
```bash
/etc/init.d/ssh start
```
You also want to start the Hadoop server unless you're running Spark in standalone mode.
```bash
start-dfs.sh
```
Finally, once Hadoop is running you can do basic things like copy files to your home directory, list files, etc like this:
```bash
hdfs dfs -cp file:$PATH .
```

From here, the first thing to do is to create the SparkContext. Note that if you run this twice without restarting the kernel, you will get an error saying that there can only be one SparkContext.

In [1]:
from pyspark import SparkContext
from pyspark import SparkConf

conf = SparkConf()
conf.setMaster('local[2]').setAppName('Tutorial')
sc = SparkContext(conf)

# Getting a Dataset

Now that everything is running, we can load in some data. I've placed the text of Moby Dick into my Hadoop home directory. This is from Project Gutenberg and is the version currently included as part of the NLTK Gutenberg corpus. Since this is just a text file, we can create a Spark RDD by just calling textFile().

In [4]:
df = sc.textFile('melville-moby_dick.txt')

Each entry in the dataset represents a line. So how many lines are there? We can just run the count() function.

In [5]:
df.count()

22924

But that's not very interesting. It turns out that the main text starts right at line 500. Let's grab the first 550 lines and then print the first 50 lines of the actual text. You may remember that the first line of Moby Dick is "Call me Ishmael."

In [45]:
lines = df.take(550)
for i in range(500,550):
    print(lines[i])


CHAPTER 1

Loomings.


Call me Ishmael.  Some years ago--never mind how long
precisely--having little or no money in my purse, and nothing
particular to interest me on shore, I thought I would sail about a
little and see the watery part of the world.  It is a way I have of
driving off the spleen and regulating the circulation.  Whenever I
find myself growing grim about the mouth; whenever it is a damp,
drizzly November in my soul; whenever I find myself involuntarily
pausing before coffin warehouses, and bringing up the rear of every
funeral I meet; and especially whenever my hypos get such an upper
hand of me, that it requires a strong moral principle to prevent me
from deliberately stepping into the street, and methodically knocking
people's hats off--then, I account it high time to get to sea as soon
as I can.  This is my substitute for pistol and ball.  With a
philosophical flourish Cato throws himself upon his sword; I quietly
take to the ship.  There is nothing surprising in thi

# Counting Words in Moby Dick

We already got the number of lines, but that is dependent on things like the intended font and page width. It would be better to count things like the number of words. This is probably the most common task in the Spark introductions that I've seen. I'll do something a bit more complicated than what is found in the introduction on the Spark website.

Because this is a prose text, there is a lot of punctuation. Here, I'll first remove all non alphanumeric characters and then get a count of each distinct word, where a "word" in this context is just a set of consecutive non-whitespace characters.

In [64]:
import re

allwords = df.map(lambda x : re.sub(r'\W',' ',x)). \
                flatMap(lambda line: line.split()).\
                filter(lambda x: x!='')

counts = allwords.map(lambda word: (word,1)).\
                  reduceByKey(lambda x,y : x+y)

As you can see, I've split the operations into two parts. One gets a list of all words, while the other takes the words and gets a count for each unique word. I actually haven't calculated anything. I'll need to call another function to tell Spark to run these functions. But, what are the functions that I've called here?

```python
map()
```
This may be familiar to you from Pandas or even from regular Python code.
Map loops over all entries in the dataset, applies the given function, and returns a new dataset with the results.

```python
flatMap()
```
This function is much like map() except it flattens things like lists into a series of individual entries.

```python
filter()
```
This may also be familiar. It gets a new dataset after removing entries failing the given function.

```python
reduceByKey()
```
Finally, there's reduceByKey(). The last map() statement turns each word into a (key,value) pair. reduceByKey() groups together all entries with the same key and then runs some function. In the lambda function that I've defined, I add the new value (y) to the current sum (x).

In [66]:
totalwords = allwords.count()
print("Total # of words found: {}".format(totalwords))
distinctwords = counts.count()
print('Total # of distinct words found: {}'.format(distinctwords))
# Alternative distinct counter:
print('Another distinct count: {}'.format(allwords.distinct().count()))

Total # of words found: 218621
Total # of distinct words found: 19226
Another distinct count: 19226


So, we see that there are 218,621 distinct "words" here and 19,226 distinct words. Of course, this isn't really the same as if we went through the text by hand. I haven't attempted to correct for capitalization. I also haven't tried to correct contractions to their original words. Something like "can't" will be corrected to "can t" here, which will be counted as two words.

# More Operations

What if we want to get the most common word? We'll just define a comparison function and run reduce().

In [67]:
def get_max(cur_max,newval):
    if cur_max[1]>=newval[1]:
        return cur_max
    return newval
top = counts.reduce(get_max )
print(top)

('the', 13721)


We can also sort the results by defining a function that returns a value to be compared.

In [68]:
sortedData = counts.sortBy(lambda x : x[1],ascending=True)
sortedData.take(20)

[('Consumptive', 1),
 ('lexicons', 1),
 ('mockingly', 1),
 ('flags', 1),
 ('HVAL', 1),
 ('Dut', 1),
 ('Ger', 1),
 ('WALW', 1),
 ('RICHARDSON', 1),
 ('LATIN', 1),
 ('WHOEL', 1),
 ('SAXON', 1),
 ('HWAL', 1),
 ('SWEDISH', 1),
 ('ICELANDIC', 1),
 ('Librarian', 1),
 ('painstaking', 1),
 ('burrower', 1),
 ('grub', 1),
 ('stalls', 1)]

We see that we get an unsorted list of some of the terms that appear only once. Many of these are capitalized or even in all caps and some aren't even English words, so we see that further analysis could change this list significantly.

In [69]:
sortedData = counts.sortBy(lambda x: x[1],ascending=False)
sortedData.take(20)

[('the', 13721),
 ('of', 6536),
 ('and', 6024),
 ('a', 4569),
 ('to', 4542),
 ('in', 3916),
 ('that', 2982),
 ('his', 2459),
 ('it', 2209),
 ('I', 2124),
 ('s', 1739),
 ('is', 1695),
 ('he', 1661),
 ('with', 1659),
 ('was', 1632),
 ('as', 1620),
 ('all', 1462),
 ('for', 1414),
 ('this', 1280),
 ('at', 1231)]

As expected, many of the most common words are basic words such as pronouns, articles, conjunctions, and prepositions. We can also sort the words by key to get an alphabetical list.

In [71]:
counts.sortByKey().take(10)

[('000', 20),
 ('1', 2),
 ('10', 4),
 ('100', 1),
 ('101', 1),
 ('102', 1),
 ('103', 1),
 ('104', 1),
 ('105', 1),
 ('106', 1)]

In [72]:
counts.sortByKey(ascending=False).take(10)

[('zoology', 1),
 ('zones', 3),
 ('zoned', 1),
 ('zone', 5),
 ('zodiac', 3),
 ('zig', 1),
 ('zephyr', 1),
 ('zeal', 2),
 ('zay', 1),
 ('zag', 1)]

Here, we can see that we have three forms of the word "zone." For many applications, we might want to run a stemmer or lemmatizer, which should combine these into a single entry.